# Machine Learning to Identify Fraud in the Enron Corpus

In [ ]:
Alexandra AMIENS
Léo ITURRIOZ
Erwan HENRY DE VILLENEUVE
Julieva COHEN-SORS

Import all the packages we need to build our predictive model 

In [1]:
import sys
import pickle
sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
import pandas as pd
import numpy as np
import sys
import pickle
import csv
import matplotlib.pyplot as plt
import pprint
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go
import warnings 
warnings.filterwarnings("ignore")

sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit
#from poi_data import*
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

from numpy import mean

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import accuracy_score, precision_score, recall_score

from plotly import tools

## Task 1: Select Feature we'll use 

In order to complete an exploratory analysis and choose which features is the best (part 3), we will use all the features in the dataset.


In [2]:
target_label = 'poi'

email_features_list = [
    'from_messages',
    'from_poi_to_this_person',
    'from_this_person_to_poi',
    'shared_receipt_with_poi',
    'to_messages',
    ]
    
financial_features_list = [
    'bonus',
    'deferral_payments',
    'deferred_income',
    'director_fees',
    'exercised_stock_options',
    'expenses',
    'loan_advances',
    'long_term_incentive',
    'other',
    'restricted_stock',
    'restricted_stock_deferred',
    'salary',
    'total_payments',
    'total_stock_value',
]

features_list = [target_label] + financial_features_list + email_features_list


#### Load the dictionary containing the dataset

In [3]:
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

### Explore csv file 

In [4]:
def make_csv(data_dict):
    """ generates a csv file from a data set"""
    fieldnames = ['name'] + data_dict.itervalues().next().keys()
    with open('data.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for record in data_dict:
            person = data_dict[record]
            person['name'] = record
            assert set(person.keys()) == set(fieldnames)
            writer.writerow(person)

### Dataset Exploration

In [5]:
print('# Exploratory Data Analysis #')
data_dict.keys()
print('Total number of data points: %d' % len(data_dict.keys()))
num_poi = 0
for name in data_dict.keys():
    if data_dict[name]['poi'] == True:
        num_poi += 1
print('Number of Persons of Interest: %d' % num_poi)
print('Number of people without Person of Interest label: %d' % (len(data_dict.keys()) - num_poi))


# Exploratory Data Analysis #
Total number of data points: 146
Number of Persons of Interest: 18
Number of people without Person of Interest label: 128


### Feature Exploration

In [6]:
all_features = data_dict['ALLEN PHILLIP K'].keys()
print('Each person has %d features available' %  len(all_features))

Each person has 21 features available


#### Evaluate dataset for completeness

In [7]:
missing_values = {}
for feature in all_features:
    missing_values[feature] = 0
for person in data_dict.keys():
    records = 0
    for feature in all_features:
        if data_dict[person][feature] == 'NaN':
            missing_values[feature] += 1
        else:
            records += 1

#### Print results of completeness analysis

In [8]:
print('Number of Missing Values for Each Feature:')
#for feature in all_features:
    #print("%s: %d" % (feature, missing_values[feature]))

for id in sorted(missing_values, key = missing_values.get, reverse = True):
        print(id, missing_values[id])

Number of Missing Values for Each Feature:
loan_advances 142
director_fees 129
restricted_stock_deferred 128
deferral_payments 107
deferred_income 97
long_term_incentive 80
bonus 64
to_messages 60
from_poi_to_this_person 60
from_messages 60
from_this_person_to_poi 60
shared_receipt_with_poi 60
other 53
salary 51
expenses 51
exercised_stock_options 44
restricted_stock 36
email_address 35
total_payments 21
total_stock_value 20
poi 0


#### We convert the dictionary values into a dataFrame

In [9]:
enron_data = pd.DataFrame.from_dict(data_dict, orient = 'index') 

## Task 2: Data exploration and deal with/remove outliers 

In [10]:
enron_data.T

,METTS MARK,BAXTER JOHN C,ELLIOTT STEVEN,CORDES WILLIAM R,HANNON KEVIN P,MORDAUNT KRISTINA M,MEYER ROCKFORD G,MCMAHON JEFFREY,HAEDICKE MARK E,PIPER GREGORY F,...,SAVAGE FRANK,IZZO LAWRENCE L,TILNEY ELIZABETH A,MARTIN AMANDA K,BUY RICHARD B,GRAMM WENDY L,CAUSEY RICHARD A,TAYLOR MITCHELL S,DONAHUE JR JEFFREY M,GLISAN JR BEN F
salary,365788,267102,170941,NaN,243293,267093,NaN,370448,374125,197091,...,NaN,85274,247338,349487,330546,NaN,415189,265214,278601,274975
to_messages,807,NaN,NaN,764,1045,NaN,232,2355,4009,1238,...,NaN,496,460,1522,3523,NaN,1892,533,865,873
deferral_payments,NaN,1295738,NaN,NaN,NaN,NaN,1848227,NaN,2157527,1130036,...,NaN,NaN,NaN,85430,649584,NaN,NaN,227449,NaN,NaN
total_payments,1061827,5634343,211725,NaN,288682,628522,1848227,4099771,3859065,1737629,...,3750,1979596,399393,8407016,2355702,119292,1868758,1092663,875760,1272284
loan_advances,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bonus,600000,1200000,350000,NaN,1500000,325000,NaN,2600000,1150000,400000,...,NaN,NaN,300000,NaN,900000,NaN,1000000,600000,800000,600000
email_address,mark.metts@enron.com,NaN,steven.elliott@enron.com,bill.cordes@enron.com,kevin.hannon@enron.com,kristina.mordaunt@enron.com,rockford.meyer@enron.com,jeffrey.mcmahon@enron.com,mark.haedicke@enron.com,greg.piper@enron.com,...,NaN,larry.izzo@enron.com,elizabeth.tilney@enron.com,a..martin@enron.com,rick.buy@enron.com,NaN,richard.causey@enron.com,mitchell.taylor@enron.com,jeff.donahue@enron.com,ben.glisan@enron.com
restricted_stock_deferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-329825,-409554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
deferred_income,NaN,-1386055,-400729,NaN,-3117011,NaN,NaN,NaN,-934484,-33333,...,-121284,NaN,-575000,NaN,-694862,NaN,-235000,NaN,-300000,NaN
total_stock_value,585062,10623258,6678735,1038185,6391065,208510,955873,1662855,803094,880290,...,NaN,5819980,1168042,2070306,3444470,NaN,2502063,3745048,1080988,778546


In [11]:
enron_data.head(10) #shows first 10 values

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
METTS MARK,365788,807,NaN,1061827,NaN,600000,mark.metts@enron.com,NaN,NaN,585062,...,38,NaN,29,1740,1,False,NaN,702,585062,NaN
BAXTER JOHN C,267102,NaN,1295738,5634343,NaN,1200000,NaN,NaN,-1386055,10623258,...,NaN,6680544,NaN,2660303,NaN,False,1586055,NaN,3942714,NaN
ELLIOTT STEVEN,170941,NaN,NaN,211725,NaN,350000,steven.elliott@enron.com,NaN,-400729,6678735,...,NaN,4890344,NaN,12961,NaN,False,NaN,NaN,1788391,NaN
CORDES WILLIAM R,NaN,764,NaN,NaN,NaN,NaN,bill.cordes@enron.com,NaN,NaN,1038185,...,10,651850,12,NaN,0,False,NaN,58,386335,NaN
HANNON KEVIN P,243293,1045,NaN,288682,NaN,1500000,kevin.hannon@enron.com,NaN,-3117011,6391065,...,32,5538001,32,11350,21,True,1617011,1035,853064,NaN
MORDAUNT KRISTINA M,267093,NaN,NaN,628522,NaN,325000,kristina.mordaunt@enron.com,NaN,NaN,208510,...,NaN,NaN,NaN,1411,NaN,False,NaN,NaN,208510,NaN
MEYER ROCKFORD G,NaN,232,1848227,1848227,NaN,NaN,rockford.meyer@enron.com,NaN,NaN,955873,...,0,493489,28,NaN,0,False,NaN,22,462384,NaN
MCMAHON JEFFREY,370448,2355,NaN,4099771,NaN,2600000,jeffrey.mcmahon@enron.com,NaN,NaN,1662855,...,58,1104054,48,297353,26,False,694862,2228,558801,NaN
HAEDICKE MARK E,374125,4009,2157527,3859065,NaN,1150000,mark.haedicke@enron.com,-329825,-934484,803094,...,180,608750,1941,52382,61,False,983346,1847,524169,NaN
PIPER GREGORY F,197091,1238,1130036,1737629,NaN,400000,greg.piper@enron.com,-409554,-33333,880290,...,61,880290,222,778,48,False,NaN,742,409554,NaN


In [12]:
enron_data.tail(10) #shows last 10 values

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
SAVAGE FRANK,NaN,NaN,NaN,3750,NaN,NaN,NaN,NaN,-121284,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,125034
IZZO LAWRENCE L,85274,496,NaN,1979596,NaN,NaN,larry.izzo@enron.com,NaN,NaN,5819980,...,28,2165172,19,1553729,5,False,312500,437,3654808,NaN
TILNEY ELIZABETH A,247338,460,NaN,399393,NaN,300000,elizabeth.tilney@enron.com,NaN,-575000,1168042,...,10,591250,19,152055,11,False,275000,379,576792,NaN
MARTIN AMANDA K,349487,1522,85430,8407016,NaN,NaN,a..martin@enron.com,NaN,NaN,2070306,...,8,2070306,230,2818454,0,False,5145434,477,NaN,NaN
BUY RICHARD B,330546,3523,649584,2355702,NaN,900000,rick.buy@enron.com,NaN,-694862,3444470,...,156,2542813,1053,400572,71,False,769862,2333,901657,NaN
GRAMM WENDY L,NaN,NaN,NaN,119292,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,119292
CAUSEY RICHARD A,415189,1892,NaN,1868758,NaN,1000000,richard.causey@enron.com,NaN,-235000,2502063,...,58,NaN,49,307895,12,True,350000,1585,2502063,NaN
TAYLOR MITCHELL S,265214,533,227449,1092663,NaN,600000,mitchell.taylor@enron.com,NaN,NaN,3745048,...,0,3181250,29,NaN,0,False,NaN,300,563798,NaN
DONAHUE JR JEFFREY M,278601,865,NaN,875760,NaN,800000,jeff.donahue@enron.com,NaN,-300000,1080988,...,188,765920,22,891,11,False,NaN,772,315068,NaN
GLISAN JR BEN F,274975,873,NaN,1272284,NaN,600000,ben.glisan@enron.com,NaN,NaN,778546,...,52,384728,16,200308,6,True,71023,874,393818,NaN


In [13]:
enron_data.shape #analyze the shape of the dataset

(146, 21)

#### Summary of the DataFrame

In [14]:
enron_data.info() 

print ("There are a total of {} people in the dataset." .format(len(enron_data.index)))
print ("Out of which {} are POI and {} Non-POI." .format(enron_data['poi'].value_counts()[True], 
                                                 enron_data['poi'].value_counts()[False]))
print ("Total number of email plus financial features are {}. 'poi' column is our label." .format(len(enron_data.columns)-1))

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   salary                     146 non-null    object
 1   to_messages                146 non-null    object
 2   deferral_payments          146 non-null    object
 3   total_payments             146 non-null    object
 4   loan_advances              146 non-null    object
 5   bonus                      146 non-null    object
 6   email_address              146 non-null    object
 7   restricted_stock_deferred  146 non-null    object
 8   deferred_income            146 non-null    object
 9   total_stock_value          146 non-null    object
 10  expenses                   146 non-null    object
 11  from_poi_to_this_person    146 non-null    object
 12  exercised_stock_options    146 non-null    object
 13  from_messages              146 non-null    object

#### descriptive statistics

In [15]:
enron_data.describe()

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
count,146,146,146,146,146,146,146,146,146,146,...,146,146,146,146,146,146,146,146,146,146
unique,95,87,40,126,5,42,112,19,45,125,...,58,102,65,93,42,2,53,84,98,18
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
freq,51,60,107,21,142,64,35,128,97,20,...,60,44,60,53,60,128,80,60,36,129


#### Specific Analysis of one of the POI

Sample data for one of the Top executives - Jeffrey Skilling

In [16]:
pprint.pprint(data_dict["SKILLING JEFFREY K"])

{'bonus': 5600000,
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'jeff.skilling@enron.com',
 'exercised_stock_options': 19250000,
 'expenses': 29336,
 'from_messages': 108,
 'from_poi_to_this_person': 88,
 'from_this_person_to_poi': 30,
 'loan_advances': 'NaN',
 'long_term_incentive': 1920000,
 'other': 22122,
 'poi': True,
 'restricted_stock': 6843672,
 'restricted_stock_deferred': 'NaN',
 'salary': 1111258,
 'shared_receipt_with_poi': 2042,
 'to_messages': 3627,
 'total_payments': 8682716,
 'total_stock_value': 26093672}


### Deal with Missing Values, NaN 

In [17]:

for feat in data_dict: 
    data_e = data_dict[feat]
    if data_e['deferred_income'] == 'NaN' :
        data_e['deferred_income'] = 0
        
    if data_e['email_address'] == 'NaN' :
        data_e['email_address'] = 0     
        
    if data_e['other'] == 'NaN' :
        data_e['other'] = 0
        
    if data_e['restricted_stock'] == 'NaN' :
        data_e['restricted_stock'] = 0    
        
    if data_e['shared_receipt_with_poi'] == 'NaN' :
        data_e['shared_receipt_with_poi'] = 0
        
    if data_e['deferral_payments'] == 'NaN' :
        data_e['deferral_payments'] = 0 
        
    if data_e['loan_advances'] == 'NaN':
        data_e['loan_advances'] = 0 
        
    if data_e['bonus'] == 'NaN' :
        data_e['bonus'] = 0
        
    if data_e['restricted_stock_deferred'] == 'NaN':
        data_e['restricted_stock_deferred'] = 0
        
    if data_e['total_stock_value'] == 'NaN' :
        data_e['total_stock_value'] = 0
        
    if data_e['expenses'] == 'NaN' :
        data_e['expenses'] = 0
        
    if data_e['exercised_stock_options'] == 'NaN' :
        data_e['exercised_stock_options'] = 0   
        
    if data_e['long_term_incentive'] == 'NaN' :
        data_e['long_term_incentive'] = 0
        
    if data_e['director_fees'] == 'NaN' :
        data_e['director_fees'] = 0
        


#### Specific Analysis of LOCKHART EUGENE E and BAXTER JOHN C 

In [18]:
pprint.pprint(data_dict["LOCKHART EUGENE E"])
pprint.pprint(data_dict["BAXTER JOHN C"])

{'bonus': 0,
 'deferral_payments': 0,
 'deferred_income': 0,
 'director_fees': 0,
 'email_address': 0,
 'exercised_stock_options': 0,
 'expenses': 0,
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 0,
 'long_term_incentive': 0,
 'other': 0,
 'poi': False,
 'restricted_stock': 0,
 'restricted_stock_deferred': 0,
 'salary': 'NaN',
 'shared_receipt_with_poi': 0,
 'to_messages': 'NaN',
 'total_payments': 'NaN',
 'total_stock_value': 0}
{'bonus': 1200000,
 'deferral_payments': 1295738,
 'deferred_income': -1386055,
 'director_fees': 0,
 'email_address': 0,
 'exercised_stock_options': 6680544,
 'expenses': 11200,
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 0,
 'long_term_incentive': 1586055,
 'other': 2660303,
 'poi': False,
 'restricted_stock': 3942714,
 'restricted_stock_deferred': 0,
 'salary': 267102,
 'shared_receipt_with_poi': 0,
 'to_messages': 'NaN',
 

Eugene Lockhart only has NaN values, so it is better to remove it as an outlier.
John Baxter has a lot of missing values especially on features we want to keep for our model.
So it also better to remove it as an outlier

### Visualization of outliers

In [19]:
# conda install -c plotly plotly chart-studio

chart_studio.tools.set_credentials_file(username='Praline96', api_key='kXkhT1CwKs4MSwsl5d3u')

# Make scatterplot before outlier removal
trace0 = go.Scatter(
    x=enron_data.salary,
    y=enron_data.bonus,
    text = enron_data.index,
    mode = 'markers'
)

# Remove Outliers
enron_data.drop(['TOTAL'], axis = 0, inplace= True)
enron_data.drop(['THE TRAVEL AGENCY IN THE PARK'], axis = 0, inplace= True)
enron_data.drop(['LOCKHART EUGENE E'], axis = 0, inplace= True)


# Make scatterplot after outliers removal
trace1 = go.Scatter(
    x=enron_data.salary,
    y=enron_data.bonus,
    text = enron_data.index,
    mode = 'markers'
)

# Layout the plots together side by side
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Before outliers removal', 'After outliers removal'))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout']['xaxis1'].update(title='salary')
fig['layout']['xaxis2'].update(title='salary')

fig['layout']['yaxis1'].update(title='bonus')
fig['layout']['yaxis2'].update(title='bonus')

py.iplot(fig)

/opt/anaconda3/lib/python3.8/site-packages/plotly/tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



We used visualization to find outliers because it is the clearest way to find them. 
When plotting salary and  bonus, we found an outlier : "TOTAL" 
We found one more outlier which is not corresponding with a name of a real person "THE TRAVEL AGENCY IN THE PARK". 
We dropped these outliers from the dataset.

In [20]:
# Make scatterplot before outlier removal
trace0 = go.Scatter(
    x=enron_data.salary,
    y=enron_data.bonus,
    text = enron_data.index,
    mode = 'markers'
)

# Remove Outliers
enron_data.drop(['LAVORATO JOHN J'], axis = 0, inplace= True)
enron_data.drop(['FREVERT MARK A'], axis = 0, inplace= True)
enron_data.drop(['BAXTER JOHN C'], axis = 0, inplace= True)
enron_data.drop(['WHALLEY LAWRENCE G'], axis = 0, inplace= True)


# Make scatterplot after outliers removal
trace1 = go.Scatter(
    x=enron_data.salary,
    y=enron_data.bonus,
    text = enron_data.index,
    mode = 'markers'
)

# Layout the plots together side by side
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Before outliers removal', 'After outliers removal'))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout']['xaxis1'].update(title='salary')
fig['layout']['xaxis2'].update(title='salary')

fig['layout']['yaxis1'].update(title='bonus')
fig['layout']['yaxis2'].update(title='bonus')

py.iplot(fig)


/opt/anaconda3/lib/python3.8/site-packages/plotly/tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



Seeing the plot "Before outliers removal" we decided to remove the extrems values(non-POI) and after analysing
the board of the company.

### Remove outliers on the data_dict

In [21]:
data_dict.pop('FREVERT MARK A')
data_dict.pop('LAVORATO JOHN J')
data_dict.pop('WHALLEY LAWRENCE G')
data_dict.pop('BAXTER JOHN C')
data_dict.pop('TOTAL')
data_dict.pop('THE TRAVEL AGENCY IN THE PARK')
data_dict.pop('LOCKHART EUGENE E')

{'salary': 'NaN',
 'to_messages': 'NaN',
 'deferral_payments': 0,
 'total_payments': 'NaN',
 'loan_advances': 0,
 'bonus': 0,
 'email_address': 0,
 'restricted_stock_deferred': 0,
 'deferred_income': 0,
 'total_stock_value': 0,
 'expenses': 0,
 'from_poi_to_this_person': 'NaN',
 'exercised_stock_options': 0,
 'from_messages': 'NaN',
 'other': 0,
 'from_this_person_to_poi': 'NaN',
 'poi': False,
 'long_term_incentive': 0,
 'shared_receipt_with_poi': 0,
 'restricted_stock': 0,
 'director_fees': 0}

## Task 3: Create new feature(s) 

### Create new copies of dataset for grading

In [22]:
my_dataset = data_dict

### Feature Engineering, new features 

In [23]:
from decimal import Decimal, getcontext

Now let's add some new features of interest to the list of the features. We will make a function to do that.

In [24]:
def add_poi_ratio(data_dict, features_list):
    """ mutates data dict to add proportion of email interaction with pois """
    fields = ['to_messages', 'from_messages',
              'from_poi_to_this_person', 'from_this_person_to_poi']
    for record in data_dict:
        person = data_dict[record]
        is_valid = True
        for field in fields:
            if person[field] == 'NaN':
                is_valid = False
        if is_valid:
            total_messages = person['to_messages'] +\
                             person['from_messages']
            poi_messages = person['from_poi_to_this_person'] +\
                           person['from_this_person_to_poi']
            person['poi_ratio'] = float(poi_messages) / total_messages
        else:
            person['poi_ratio'] = 'NaN'
    features_list += ['poi_ratio']

def add_fraction_to_poi(data_dict, features_list):
    """ mutates data dict to add proportion of email fraction_to_poi """
    fields = ['from_messages', 'from_this_person_to_poi']
    for record in data_dict:
        person = data_dict[record]
        is_valid = True
        for field in fields:
            if person[field] == 'NaN':
                is_valid = False
        if is_valid:
            total_messages = person['from_messages']
            poi_messages =   person['from_this_person_to_poi']
            person['fraction_to_poi'] = float(poi_messages) / total_messages
        else:
            person['fraction_to_poi'] = 'NaN'
    features_list += ['fraction_to_poi']

def add_fraction_from_poi(data_dict, features_list):
    """ mutates data dict to add proportion of email fraction_from_poi """
    fields = ['to_messages', 'from_poi_to_this_person']
    for record in data_dict:
        person = data_dict[record]
        is_valid = True
        for field in fields:
            if person[field] == 'NaN':
                is_valid = False
        if is_valid:
            total_messages = person['to_messages']
            poi_messages =   person['from_poi_to_this_person']
            person['fraction_from_poi'] = float(poi_messages) / total_messages
        else:
            person['fraction_from_poi'] = 'NaN'
    features_list += ['fraction_from_poi']
    
def add_bonus_to_salary(data_dict, features_list):
    fields = ['bonus', 'salary']
    for record in data_dict:
        person = data_dict[record]
        is_valid = True
        for field in fields:
            if person[field] == 'NaN':
                is_valid = False
        if is_valid:
           Total_bonus = person['bonus']
           Total_salary = person['salary']
           person['bonus_to_salary'] = float(Total_bonus) / Total_salary
        else:
            person['bonus_to_salary'] = 'NaN'
    features_list += ['bonus_to_salary']

def add_bonus_to_total(data_dict, features_list):
    fields = ['bonus', 'total_payments']
    for record in data_dict:
        person = data_dict[record]
        is_valid = True
        for field in fields:
            if person[field] == 'NaN':
                is_valid = False
        if is_valid:
           Total_bonus2 = person['bonus']
           Total_payments2 = person['total_payments']
           person['bonus_to_total'] = float(Total_bonus2) / Total_payments2
        else:
            person['bonus_to_total'] = 'NaN'
    features_list += ['bonus_to_total']   

my_feature_list = features_list

#Adding them to the features list
add_poi_ratio(data_dict, my_feature_list)
add_fraction_to_poi(data_dict, my_feature_list)
add_fraction_from_poi(data_dict, my_feature_list)
add_bonus_to_salary(data_dict, my_feature_list)
add_bonus_to_total(data_dict, my_feature_list)
print (my_feature_list) 

['poi', 'bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'exercised_stock_options', 'expenses', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'total_payments', 'total_stock_value', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'shared_receipt_with_poi', 'to_messages', 'poi_ratio', 'fraction_to_poi', 'fraction_from_poi', 'bonus_to_salary', 'bonus_to_total']


We created 3 features with the email_features_list and 2 features with the financial_features_list
* poi_ratio
* fraction_to_poi
* fraction_from_poi
* bonus_to_salary
* bonus_to_total
 

### Create new copies of feature list for grading

In [25]:
my_feature_list = features_list

### Get K-best features

In [26]:
num_features = 10

### Function using SelectKBest

In [27]:
def get_k_best(data_dict, features_list, k):
    """ runs scikit-learn's SelectKBest feature selection
        returns dict where keys=features, values=scores
    """
    data = featureFormat(data_dict, features_list)
    labels, features = targetFeatureSplit(data)

    k_best = SelectKBest(k=k)
    k_best.fit(features, labels)
    scores = k_best.scores_
    print(scores)
    unsorted_pairs = zip(features_list[1:], scores)
    sorted_pairs = list(reversed(sorted(unsorted_pairs, key=lambda x: x[1])))
    k_best_features = dict(sorted_pairs[:k])
    print ("{0} best features: {1}\n".format(k, k_best_features.keys(), scores))
    return k_best_features

In [28]:
best_features = get_k_best(my_dataset, my_feature_list, num_features)

my_feature_list = [target_label] + list(set(best_features.keys()))

[35.7816913   0.06233576 16.95719491  2.20445373 26.5605672   6.27032312
  7.00482086 12.56239963  7.79759949 10.37309462  0.06764556 25.10415132
  9.53332413 26.46587154  0.13885137 11.74247165  3.34065915 12.09832639
  2.27518447  5.76594457 16.27553035  3.43989738 16.78134655 20.61205177]
10 best features: dict_keys(['bonus', 'exercised_stock_options', 'total_stock_value', 'salary', 'bonus_to_total', 'deferred_income', 'bonus_to_salary', 'fraction_to_poi', 'long_term_incentive', 'shared_receipt_with_poi'])



In [29]:
print ("{0} selected features: {1}\n".format(len(my_feature_list) - 1, my_feature_list[1:]))

10 selected features: ['total_stock_value', 'bonus_to_total', 'bonus', 'shared_receipt_with_poi', 'long_term_incentive', 'salary', 'deferred_income', 'exercised_stock_options', 'bonus_to_salary', 'fraction_to_poi']



### Extract the features specified in features_list

In [30]:
data = featureFormat(my_dataset, my_feature_list,sort_keys = True)

#### split into labels and features

In [31]:
labels, features = targetFeatureSplit(data)

### Scale features via min-max

In [32]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
features = scaler.fit_transform(features)


## Task 4 Using Algorithms

### Gaussian Naive Bayes Classifier

In [33]:
from sklearn.naive_bayes import GaussianNB
g_clf = GaussianNB()

### DecisionTreeClassifier

In [34]:
from sklearn.tree import DecisionTreeClassifier
d_clf = DecisionTreeClassifier(random_state=42)

### LogisticRegression

In [35]:
from sklearn.linear_model import LogisticRegression

l_clf = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(C=1e-08, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, 
max_iter=100, multi_class='ovr', penalty='l2', random_state=42, solver='liblinear', tol=0.001, verbose=0))])


### KMeans

In [36]:
from sklearn.cluster import KMeans
k_clf = KMeans(n_clusters=2, tol=0.001)

### RandomForestClassifier

In [37]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(max_depth = 5,max_features = 'sqrt',n_estimators = 10, random_state = 42)


### Gradient Boosting Classifier

In [38]:
from sklearn.ensemble  import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100,random_state = 42)

### Evaluate functions

In [39]:
def evaluate_clf(clf, features, labels, num_iters=1000, test_size=0.3):
    print (clf)
    accuracy = []
    precision = []
    recall = []
    first = True
    for trial in range(num_iters):
        features_train, features_test, labels_train, labels_test =\
            train_test_split(features, labels, test_size=test_size)
        clf.fit(features_train, labels_train)
        predictions = clf.predict(features_test)
        accuracy.append(accuracy_score(labels_test, predictions))
        precision.append(precision_score(labels_test, predictions))
        recall.append(recall_score(labels_test, predictions))
        if trial % 10 == 0:
            if first:
                sys.stdout.write('\nProcessing')
            sys.stdout.write('.')
            sys.stdout.flush()
            first = False

    print ("done.\n")
    print ("precision: {}".format(mean(precision)))
    print ("recall:    {}".format(mean(recall)))
    return mean(precision), mean(recall)

In [40]:
evaluate_clf(g_clf, features, labels)
evaluate_clf(d_clf, features, labels)
evaluate_clf(l_clf, features, labels)
evaluate_clf(k_clf, features, labels)
evaluate_clf(rf_clf, features, labels)
evaluate_clf(gb_clf, features, labels)

GaussianNB()

Processing....................................................................................................done.

precision: 0.4629273698523699
recall:    0.4049765873015873
DecisionTreeClassifier(random_state=42)

Processing....................................................................................................done.

precision: 0.42463833388833394
recall:    0.3971323454323454
Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 LogisticRegression(C=1e-08, multi_class='ovr', random_state=42,
                                    solver='liblinear', tol=0.001))])

Processing....................................................................................................done.

precision: 0.39017214835817776
recall:    0.5921176406926406
KMeans(n_clusters=2, tol=0.001)

Processing....................................................................................................done.

precision: 0.34572295041210005
re

(0.43576684704184704, 0.33185082972582974)

The two best models for our case seems to be the LogisticRegression and the DecisionTree 

## Task 5: Tune your classifier 

using our testing script. Check the tester.py script in the final project
folder for details on the evaluation method, especially the test_classifier
function. Because of the small size of the dataset, the script uses
stratified shuffle split cross validation. For more info: 
http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

We select the two best models from the evaluate function (DecisionTree and LogisticRegression), in order to tune the hyperparameters. 
We create a pipeline to run a GridSearch on the select K best as to find the best features selection number.

### Logistic Regression

In [41]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV


In [42]:
n_features = np.arange(1, 20)
my_feature_list = features_list 
data = featureFormat(my_dataset, my_feature_list,sort_keys = True)
labels, features = targetFeatureSplit(data)
# Set a pipeline with the features selected previously and classify them
pipe_k = Pipeline([
    ('scaler', StandardScaler()),
    ('select_features', SelectKBest()),
    ('classifier', LogisticRegression())])
param_grid = [
    {
        'select_features__k': n_features
    }
]

# We use the GridSearchCV tool in order to automatize the procedure and find the best number of features
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
k_lcf= GridSearchCV(pipe_k, param_grid=param_grid, scoring='f1', cv = 10)
k_lcf.fit(features, labels)




GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('select_features', SelectKBest()),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'select_features__k': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])}],
             scoring='f1')

In [43]:
k_lcf.best_score_
k_lcf.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('select_features', SelectKBest(k=6)),
                ('classifier', LogisticRegression())])

In [44]:
# Create new copies of feature list for grading
my_feature_list = features_list

# Get K-best features
num_features = 6
# Functio using SelectKBest
def get_k_best(data_dict, features_list, k):
    """ runs scikit-learn's SelectKBest feature selection
        returns dict where keys=features, values=scores
    """
    data = featureFormat(data_dict, features_list)
    labels, features = targetFeatureSplit(data)

    k_best = SelectKBest(k='all')
    k_best.fit(features, labels)
    scores = k_best.scores_
    print(scores)
    unsorted_pairs = zip(features_list[1:], scores)
    sorted_pairs = list(reversed(sorted(unsorted_pairs, key=lambda x: x[1])))
    k_best_features = dict(sorted_pairs[:k])
    print ("{0} best features: {1}\n".format(k, k_best_features.keys(), scores))
    return k_best_features


best_features = get_k_best(my_dataset, my_feature_list, num_features)

my_feature_list = [target_label] + list(set(best_features.keys()))

[35.7816913   0.06233576 16.95719491  2.20445373 26.5605672   6.27032312
  7.00482086 12.56239963  7.79759949 10.37309462  0.06764556 25.10415132
  9.53332413 26.46587154  0.13885137 11.74247165  3.34065915 12.09832639
  2.27518447  5.76594457 16.27553035  3.43989738 16.78134655 20.61205177]
6 best features: dict_keys(['bonus', 'exercised_stock_options', 'total_stock_value', 'salary', 'bonus_to_total', 'deferred_income'])



In [45]:
best_features = get_k_best(my_dataset, my_feature_list, num_features)
my_feature_list = [target_label] + list(set(best_features.keys()))

[25.23151365 19.49852458 34.10091227 23.67763405 16.20440327 25.35659192]
6 best features: dict_keys(['bonus', 'exercised_stock_options', 'total_stock_value', 'salary', 'bonus_to_total', 'deferred_income'])



In [46]:
data = featureFormat(my_dataset, my_feature_list,sort_keys = True)
labels, features = targetFeatureSplit(data)

In [47]:
pipe_log = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())])

### Define models and parameters

In [48]:
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ["l1","l2","elasticnet","none"]
c_values = np.logspace(-4, 4, 50)
class_weight=['balanced',None]
multi_class=["ovr"]

# define grid search
grid = dict(classifier__solver=solvers,classifier__penalty=penalty,classifier__C=c_values,classifier__class_weight=class_weight,classifier__multi_class=multi_class)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, train_size=0.7,random_state=42)
grid_search = GridSearchCV(estimator=pipe_log, param_grid=grid, n_jobs=-1, cv=cv,scoring = 'f1')
grid_result = grid_search.fit(features, labels)

In [49]:
grid_result.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 LogisticRegression(C=0.0001, multi_class='ovr',
                                    solver='liblinear'))])

In [50]:
grid_result.best_params_

{'classifier__C': 0.0001,
 'classifier__class_weight': None,
 'classifier__multi_class': 'ovr',
 'classifier__penalty': 'l2',
 'classifier__solver': 'liblinear'}

In [51]:
clf_rl=Pipeline(steps=[('std_slc', StandardScaler()),
                ('logistic_Reg',
                 LogisticRegression(C=0.0001,
                                    class_weight=None, multi_class='ovr',penalty= 'l2',
                                    solver='liblinear'))])

In [52]:
evaluate_clf(clf_rl,features,labels)

Pipeline(steps=[('std_slc', StandardScaler()),
                ('logistic_Reg',
                 LogisticRegression(C=0.0001, multi_class='ovr',
                                    solver='liblinear'))])

Processing....................................................................................................done.

precision: 0.5472570707070707
recall:    0.4316101454101454


(0.5472570707070707, 0.4316101454101454)

### Test of our model

In [53]:
pickle.dump(clf_rl, open("../final_project/my_classifier.pkl", "wb"))
pickle.dump(my_dataset, open("../final_project/my_dataset.pkl", "wb"))
pickle.dump(my_feature_list, open("../final_project/my_feature_list.pkl", "wb"))

In [54]:
import pickle
import sys
sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit

from sklearn.model_selection import StratifiedShuffleSplit


PERF_FORMAT_STRING = "\
\tAccuracy: {:>0.{display_precision}f}\tPrecision: {:>0.{display_precision}f}\t\
Recall: {:>0.{display_precision}f}\tF1: {:>0.{display_precision}f}\tF2: {:>0.{display_precision}f}"
RESULTS_FORMAT_STRING = "\tTotal predictions: {:4d}\tTrue positives: {:4d}\tFalse positives: {:4d}\
\tFalse negatives: {:4d}\tTrue negatives: {:4d}"

def test_classifier(clf_rl, dataset, feature_list, folds = 1000):
    data = featureFormat(dataset, feature_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    cv = StratifiedShuffleSplit(n_splits=folds, random_state=42)
    
    true_negatives = 0
    false_negatives = 0
    true_positives = 0
    false_positives = 0
    for train_idx, test_idx in cv.split(features, labels):
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []
        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )
        
        ### fit the classifier using training set, and test on test set
        clf_rl.fit(features_train, labels_train)
        predictions = clf_rl.predict(features_test)
        for prediction, truth in zip(predictions, labels_test):
            if prediction == 0 and truth == 0:
                true_negatives += 1
            elif prediction == 0 and truth == 1:
                false_negatives += 1
            elif prediction == 1 and truth == 0:
                false_positives += 1
            elif prediction == 1 and truth == 1:
                true_positives += 1
            else:
                print ("Warning: Found a predicted label not == 0 or 1.")
                print ("All predictions should take value 0 or 1.")
                print ("Evaluating performance for processed predictions:")
                break
    try:
        total_predictions = true_negatives + false_negatives + false_positives + true_positives
        accuracy = 1.0*(true_positives + true_negatives)/total_predictions
        precision = 1.0*true_positives/(true_positives+false_positives)
        recall = 1.0*true_positives/(true_positives+false_negatives)
        f1 = 2.0 * true_positives/(2*true_positives + false_positives+false_negatives)
        f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)
        print (clf_rl)
        print (PERF_FORMAT_STRING.format(accuracy, precision, recall, f1, f2, display_precision = 5))
        print (RESULTS_FORMAT_STRING.format(total_predictions, true_positives, false_positives, false_negatives, true_negatives))
        print ("")
    except:
        print ("Got a divide by zero when trying out:", clf_rl)
        print ("Precision or recall may be undefined due to a lack of true positive predicitons.")

CLF_PICKLE_FILENAME = "my_classifier.pkl"
DATASET_PICKLE_FILENAME = "my_dataset.pkl"
FEATURE_LIST_FILENAME = "my_feature_list.pkl"

def dump_classifier_and_data(clf_rl, dataset, feature_list):
    with open(CLF_PICKLE_FILENAME, "wb") as clf_rl_outfile:
        pickle.dump(clf_rl, clf_rl_outfile)
    with open(DATASET_PICKLE_FILENAME, "wb") as dataset_outfile:
        pickle.dump(dataset, dataset_outfile)
    with open(FEATURE_LIST_FILENAME, "wb") as featurelist_outfile:
        pickle.dump(feature_list, featurelist_outfile)

def load_classifier_and_data():
    with open(CLF_PICKLE_FILENAME, "rb") as clf_rl_infile:
        clf_rl = pickle.load(clf_rl_infile)
    with open(DATASET_PICKLE_FILENAME, "rb") as dataset_infile:
        dataset = pickle.load(dataset_infile)
    with open(FEATURE_LIST_FILENAME, "rb") as featurelist_infile:
        feature_list = pickle.load(featurelist_infile)
    return clf_rl, dataset, feature_list

def main():
    ### load up student's classifier, dataset, and feature_list
    clf_rl, dataset, feature_list = load_classifier_and_data()
    ### Run testing script
    test_classifier(clf_rl, dataset, feature_list)

if __name__ == '__main__':
    main()

Pipeline(steps=[('std_slc', StandardScaler()),
                ('logistic_Reg',
                 LogisticRegression(C=0.0001, multi_class='ovr',
                                    solver='liblinear'))])
	Accuracy: 0.86450	Precision: 0.53169	Recall: 0.43200	F1: 0.47669	F2: 0.44883
	Total predictions: 14000	True positives:  864	False positives:  761	False negatives: 1136	True negatives: 11239



### Decision Tree

We identified that the Decision tree is our best model so we would like to tune the hyperparameters in order to improve our results. We will analyze the best number of features to use on our model.

In [55]:
from sklearn.model_selection import GridSearchCV
import numpy as np

In [56]:
n_features = np.arange(1, 20)
my_feature_list = features_list 
data = featureFormat(my_dataset, my_feature_list,sort_keys = True)
labels, features = targetFeatureSplit(data)
# Set a pipeline with the features selected previously and classify them
pipe_k1 = Pipeline([
    ('select_features', SelectKBest()),
    ('classifier',DecisionTreeClassifier())])
param_grid = [
    {
        'select_features__k': n_features
    }
]


# We use the GridSearchCV tool in order to automatize the procedure and find the best number of features
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
k_clf= GridSearchCV(pipe_k1, param_grid=param_grid, scoring='f1', cv = cv)
k_clf.fit(features, labels)

# We use the GridSearchCV tool in order to automatize the procedure and find the best number of features
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
k_lcf= GridSearchCV(pipe_k, param_grid=param_grid, scoring='f1', cv = 10)
k_lcf.fit(features, labels)



GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('select_features', SelectKBest()),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'select_features__k': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])}],
             scoring='f1')

In [57]:
k_clf.best_score_

0.4333111333111333

In [58]:
k_clf.best_score_
k_clf.best_estimator_

Pipeline(steps=[('select_features', SelectKBest(k=12)),
                ('classifier', DecisionTreeClassifier())])

In [59]:
num_features=12

In [60]:
best_features = get_k_best(my_dataset, my_feature_list, num_features)
my_feature_list = [target_label] + list(set(best_features.keys()))

[35.7816913   0.06233576 16.95719491  2.20445373 26.5605672   6.27032312
  7.00482086 12.56239963  7.79759949 10.37309462  0.06764556 25.10415132
  9.53332413 26.46587154  0.13885137 11.74247165  3.34065915 12.09832639
  2.27518447  5.76594457 16.27553035  3.43989738 16.78134655 20.61205177]
12 best features: dict_keys(['bonus', 'exercised_stock_options', 'total_stock_value', 'salary', 'bonus_to_total', 'deferred_income', 'bonus_to_salary', 'fraction_to_poi', 'long_term_incentive', 'shared_receipt_with_poi', 'from_poi_to_this_person', 'restricted_stock'])



In [61]:
data = featureFormat(my_dataset, my_feature_list,sort_keys = True)
labels, features = targetFeatureSplit(data)

In [62]:
clf_parameters = { 'criterion': ['gini', 'entropy'],
                   'max_depth': [None, 1, 2, 4, 5, 10, 15, 20],
                   'min_samples_split': [2, 4, 6, 8, 10, 20, 30, 40],
                   'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 10, 20, 30] }

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=42)
clf = GridSearchCV(DecisionTreeClassifier(), param_grid = clf_parameters, cv = cv, scoring = 'f1')
clf.fit(features,labels)

clf.best_estimator_

clf.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 4}

In [63]:
clf=DecisionTreeClassifier(criterion= 'gini',
 max_depth = 10,
 min_samples_leaf = 1,
 min_samples_split = 4)


In [64]:
evaluate_clf(clf,features,labels)

DecisionTreeClassifier(max_depth=10, min_samples_split=4)

Processing....................................................................................................done.

precision: 0.3730664143862673
recall:    0.35953639971139967


(0.3730664143862673, 0.35953639971139967)

### Test of our model

In [65]:
pickle.dump(clf, open("../final_project/my_classifier.pkl", "wb"))
pickle.dump(my_dataset, open("../final_project/my_dataset.pkl", "wb"))
pickle.dump(my_feature_list, open("../final_project/my_feature_list.pkl", "wb"))

In [66]:
import pickle
import sys
sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit

from sklearn.model_selection import StratifiedShuffleSplit


PERF_FORMAT_STRING = "\
\tAccuracy: {:>0.{display_precision}f}\tPrecision: {:>0.{display_precision}f}\t\
Recall: {:>0.{display_precision}f}\tF1: {:>0.{display_precision}f}\tF2: {:>0.{display_precision}f}"
RESULTS_FORMAT_STRING = "\tTotal predictions: {:4d}\tTrue positives: {:4d}\tFalse positives: {:4d}\
\tFalse negatives: {:4d}\tTrue negatives: {:4d}"

def test_classifier(clf, dataset, feature_list, folds = 1000):
    data = featureFormat(dataset, feature_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    cv = StratifiedShuffleSplit(n_splits=folds, random_state=42)
    
    true_negatives = 0
    false_negatives = 0
    true_positives = 0
    false_positives = 0
    for train_idx, test_idx in cv.split(features, labels):
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []
        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )
        
        ### fit the classifier using training set, and test on test set
        clf.fit(features_train, labels_train)
        predictions = clf.predict(features_test)
        for prediction, truth in zip(predictions, labels_test):
            if prediction == 0 and truth == 0:
                true_negatives += 1
            elif prediction == 0 and truth == 1:
                false_negatives += 1
            elif prediction == 1 and truth == 0:
                false_positives += 1
            elif prediction == 1 and truth == 1:
                true_positives += 1
            else:
                print ("Warning: Found a predicted label not == 0 or 1.")
                print ("All predictions should take value 0 or 1.")
                print ("Evaluating performance for processed predictions:")
                break
    try:
        total_predictions = true_negatives + false_negatives + false_positives + true_positives
        accuracy = 1.0*(true_positives + true_negatives)/total_predictions
        precision = 1.0*true_positives/(true_positives+false_positives)
        recall = 1.0*true_positives/(true_positives+false_negatives)
        f1 = 2.0 * true_positives/(2*true_positives + false_positives+false_negatives)
        f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)
        print (clf)
        print (PERF_FORMAT_STRING.format(accuracy, precision, recall, f1, f2, display_precision = 5))
        print (RESULTS_FORMAT_STRING.format(total_predictions, true_positives, false_positives, false_negatives, true_negatives))
        print ("")
    except:
        print ("Got a divide by zero when trying out:", clf)
        print ("Precision or recall may be undefined due to a lack of true positive predicitons.")

CLF_PICKLE_FILENAME = "my_classifier.pkl"
DATASET_PICKLE_FILENAME = "my_dataset.pkl"
FEATURE_LIST_FILENAME = "my_feature_list.pkl"

def dump_classifier_and_data(clf, dataset, feature_list):
    with open(CLF_PICKLE_FILENAME, "wb") as clf_outfile:
        pickle.dump(clf, clf_outfile)
    with open(DATASET_PICKLE_FILENAME, "wb") as dataset_outfile:
        pickle.dump(dataset, dataset_outfile)
    with open(FEATURE_LIST_FILENAME, "wb") as featurelist_outfile:
        pickle.dump(feature_list, featurelist_outfile)

def load_classifier_and_data():
    with open(CLF_PICKLE_FILENAME, "rb") as clf_infile:
        clf = pickle.load(clf_infile)
    with open(DATASET_PICKLE_FILENAME, "rb") as dataset_infile:
        dataset = pickle.load(dataset_infile)
    with open(FEATURE_LIST_FILENAME, "rb") as featurelist_infile:
        feature_list = pickle.load(featurelist_infile)
    return clf, dataset, feature_list

def main():
    ### load up student's classifier, dataset, and feature_list
    clf, dataset, feature_list = load_classifier_and_data()
    ### Run testing script
    test_classifier(clf, dataset, feature_list)

if __name__ == '__main__':
    main()

DecisionTreeClassifier(max_depth=10, min_samples_split=4)
	Accuracy: 0.84857	Precision: 0.46678	Recall: 0.42150	F1: 0.44298	F2: 0.42984
	Total predictions: 14000	True positives:  843	False positives:  963	False negatives: 1157	True negatives: 11037



Comparing the results after parameter tunning we discovered that Logistic Regression had a 
better performance than Decision Tree. We were skeptical about the difference of results for the evaluate function of the logistic Regression and its tester. The Decision Tree model seems to be the more consistent between both. Considering the latter, we would rather select the Decision tree Model. 

In [67]:
pickle.dump(clf, open("../final_project/my_classifier.pkl", "wb"))
pickle.dump(my_dataset, open("../final_project/my_dataset.pkl", "wb"))
pickle.dump(my_feature_list, open("../final_project/my_feature_list.pkl", "wb"))

## Task 6: Dump your classifier, dataset, and features_list so anyone can

Check your results. You do not need to change anything below, but make sure
that the version of poi_id.py that you submit can be run on its own and
generates the necessary .pkl files for validating your results.

In [68]:
dump_classifier_and_data(clf, my_dataset, features_list)
